In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pylab as pylab
from datetime import datetime
import datetime as datet
import os
import sys
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller

import astropy.stats as ast_stats

import pyflux as pf

import traces

import warnings
warnings.filterwarnings('ignore')
import tqdm

from sklearn.metrics import mean_absolute_error, mean_squared_error

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from scipy.optimize import minimize


from pykalman import KalmanFilter

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
init_notebook_mode(connected = True)

from sklearn.model_selection import TimeSeriesSplit

from scipy import integrate

from numpy import array, arange, abs as np_abs
from numpy.fft import rfft, rfftfreq

import stldecompose

plt.rcParams['figure.figsize'] = (10, 8)

import scipy

In [ ]:
def plotly_df(df, title = ''):
    data = []
    column = 'cpu'
    trace = go.Scatter(
        x = df.index,
        y = df.values,
        mode = 'lines',
        name = column
    )
    data.append(trace)

    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

In [ ]:
def load_files_from_folder(files_name, col_name, root='./../../Downloads/ciscoSwitch/attributes/'):
    datas = []
    for file in os.listdir(root):
        if os.path.isdir(os.path.join(root, file)):
            ip = file
            for fl in os.listdir(os.path.join(root, file)):
                cols = ['timestamp', col_name]
                if files_name in fl:
                    df = pd.read_csv(
                        os.path.join(root,file,fl),
                        sep=',',
                        encoding='utf-8',
                        dtype='str',
                        names=cols
                    )
                    df['ip'] = ip
                    datas.append(df)
    return pd.concat(datas, ignore_index=True, axis=0)

In [ ]:
def split_date_and_set_week(df, col_name):
    mapping = {0:'mon', 1:'tue', 2:'wed', 3:'thu', 4:'fri', 5:'sat', 6:'sun'}
    weeknum_mapping = {
        range(1,8):'0',
        range(8,15):'1',
        range(15,22):'2',
        range(22,29):'3',
        range(29,33):'4'
    }
    
    def set_weeknum(day):
        for k,v in weeknum_mapping.items():
            if day in k:
                return v
    
    df.timestamp = df.timestamp.astype(int)
    df['date'] = df.timestamp.apply(lambda x: datetime.fromtimestamp(x/1000000))
    df['year'] = df.date.apply(lambda x: x.year)
    df['month'] = df.date.apply(lambda x: x.month)
    df['day'] = df.date.apply(lambda x: x.day)
    df['hour'] = df.date.apply(lambda x: x.hour)
    df['minute'] = df.date.apply(lambda x: x.minute)
    df['second'] = df.date.apply(lambda x: x.second)
    df['weekday'] = df.date.apply(lambda x: mapping[x.weekday()])
    df['weeknum'] = df.day.apply(set_weeknum)
    df['weekend'] = df.weekday.apply(lambda x: 1 if x == 'sat' or x == 'sun' else 0)
    df['busy'] = df.hour.apply(lambda x: 1 if 9<=x<18 else 0)
    try:
        df[col_name] = df[col_name].astype(int)
    except:
        df[col_name] = df[col_name].astype(float)
    return df

In [ ]:
def select_data(files_name, col_name):
    df = load_files_from_folder(files_name=files_name, col_name=col_name)
    df = split_date_and_set_week(df, col_name)
    return df

In [ ]:
df = select_data('cpuUtil', 'cpu')

In [ ]:
df.head()

In [ ]:
cc = df[df.ip == '10.10.245.12']

In [ ]:
ts = pd.Series(data = cc['cpu'].values, index=cc['date'].values)

In [ ]:
ts.head(20)

In [ ]:
ts_resampled_15 = ts.resample('1T')

In [ ]:
ts_resampled_15.shape

In [ ]:
ts_resampled_15.isnull().sum()

In [ ]:
ts_resampled_15.head(10)

In [ ]:
mn = ts_resampled_15.index.min()
mx = ts_resampled_15.index.max()

In [ ]:
ts_resampled_15.shape

In [ ]:
60*24

In [ ]:
mn + datet.timedelta(days=1)

In [ ]:
st = pd.DataFrame()
st['date'] = ts_resampled_15.index.values
st['val'] = ts_resampled_15.values

In [ ]:
# st.head(10)

In [ ]:
beg = mn
end = mx
counter = mn + datet.timedelta(days=1)
fd = pd.DataFrame()
cols = []
cnt = []
while counter < mx:
    cols.append('{}-{}'.format(beg, counter))
    cnt.append(st.loc[
        (st.date >= beg)&
        (st.date < counter), 'val'].isnull().sum())
    beg = counter
    counter +=  datet.timedelta(days=1)

In [ ]:
fd['day'] = cols
fd['nan_count'] = cnt

In [ ]:
fd.head()

In [ ]:
tt = fd

In [ ]:
plt.figure(figsize=(17,9))
plt.plot(tt.nan_count)
plt.xticks(range(len(tt.day)), tt.day, rotation=90);

In [ ]:
tt = fd.sort_values(by='nan_count',ascending=False).reset_index(drop=True)

In [ ]:
plt.plot(tt.nan_count)

In [ ]:
date = cc['date']

In [ ]:
date.head()

In [ ]:
def aggregate_nan_by_week(df, col):
    
    

In [ ]:
df.ip.unique()[:5]

In [ ]:
sub_df = df[df.ip == '10.10.245.12']

In [ ]:
ts = pd.Series(data=sub_df.cpu.values, index=sub_df['date'].values)

In [ ]:
print(ts.index[0])
print(ts.index[-1])

In [ ]:
plt.scatter(ts.index.values, ts.values)

In [ ]:
ts.value_counts()

In [ ]:
autocorrelation_plot(ts)

In [ ]:
ts_mean_3 = ts.rolling(50).mean()

In [ ]:
autocorrelation_plot(ts_mean_3.dropna()[:1000])

# Посмотрим на количество измерений и уникальность значений в измерениях

In [ ]:
def get_unique_stat(df, col_name):
    ip = []
    n = []
    nunique = []
    for k,v in df.groupby(['ip']):
        ip.append(k)
        n.append(v['timestamp'].count())
        nunique.append(v[col_name].unique().shape[0])

    unique_stat = pd.DataFrame()
    unique_stat['ip'] = ip
    unique_stat['n'] = n
    unique_stat['nunique'] = nunique
    unique_stat['ratio'] = unique_stat['nunique']/unique_stat['n']
    return unique_stat

In [ ]:
un = get_unique_stat(df, 'out')

In [ ]:
un.head()

In [ ]:
sns.distplot(un['ratio'].values)

In [ ]:
def plot_valid_invalid(un):
    print(un['nunique'].median())
    print(un['nunique'].mean())
    print()
    print(un['n'].median())
    print(un['n'].mean())

    clazz = (un['n'] >= un['n'].quantile(.25)) & \
            (un['nunique'] >= un['nunique'].quantile(.25)) & \
            (un['n'] <= un['n'].quantile(.75)) & \
            (un['nunique'] <= un['nunique'].quantile(.75))
    unn = un.copy(deep=True)
    unn['class'] = clazz.astype(int).values

    sns.lmplot(data=unn, x='n', y='nunique', hue='class', size=8, aspect=2, legend=False, scatter=True, logistic=False)
    plt.xlabel(u'Количество измерений')
    plt.ylabel(u'Количество уникальных значений в измерениях')
    plt.axvline(un['n'].quantile(.25), color='g', linestyle='--', label='n quantile(.25)')
    plt.axvline(un['n'].quantile(.75), color='g', linestyle='--', label='n quantile(.75)')
    plt.axhline(un['nunique'].quantile(.25), color='r', linestyle='--', label='nunique quantile(.25)')
    plt.axhline(un['nunique'].quantile(.75), color='r', linestyle='--', label='nunique quantile(.75)')
    plt.legend()
    
    return unn

In [ ]:
unn = plot_valid_invalid(un)

In [ ]:
g = sns.JointGrid(x=un['nunique'], y=un['n'], size=8)
g.fig.suptitle('Совместный график распределения уникальных значений и количества показаний по атрибуту outUtil')

g = g.plot_joint(plt.scatter, edgecolor="white", color='b', alpha=.4)
_ = g.ax_marg_x.hist(un['nunique'], color='r', bins=30, alpha=.2)
_ = g.ax_marg_y.hist(un['n'],  orientation="horizontal", bins=30, color='g', alpha=.2)
# g = g.annotate(scipy.stats.pearsonr)
g.savefig('./../Desktop/memUtil_2.png')

## Если посмотреть на процент значений больше или меньше определенных квантилей по всем устройствам

In [ ]:
def high_and_low_quantile(df, col_name):
    ix = []
    val = []
    for k, v in df.groupby(['ip']):
        vv = v.loc[v[col_name] > v[col_name].quantile(.75), col_name].count()/v[col_name].count()*100
        ix.append(k)
        val.append(vv)

    quantile_75 = pd.Series(data=val, index=ix)

    plt.figure(figsize=(17,8))
    plt.title(u'Процент значений больше квантиля в .75 по каждому устройству на всем времени съема показаний')
    quantile_75.sort_values().plot(label=u'значения в процентах')
    n = quantile_75.count()
    plt.grid(True)
    plt.axhline(y=quantile_75.mean()+1.96*quantile_75.std(), color='green', linestyle='--')
    plt.axhline(y=quantile_75.mean()-1.96*quantile_75.std(), color='green', linestyle='--')

    threshold = quantile_75.mean()+1.96*quantile_75.std()
    plt.axvline(x =n -(quantile_75.sort_values() > threshold).sum(),
            linestyle='--')

    plt.legend()

    print(n -(quantile_75.sort_values() > threshold).sum())
    
    
    ix = []
    val = []
    for k, v in df.groupby(['ip']):
        vv = v.loc[v[col_name] < v[col_name].quantile(.25), col_name].count()/v[col_name].count()*100
        ix.append(k)
        val.append(vv)

    quantile_25 = pd.Series(data=val, index=ix)
    n = quantile_25.count()
    plt.figure(figsize=(17,8))
    plt.title(u'Процент значений меньше квантиля в .25 по каждому устройству на всем времени съема показаний')
    quantile_25.sort_values().plot(label=u'значения в процентах')
    plt.grid(True)
    plt.axhline(y=quantile_25.mean()+1.96*quantile_25.std(), color='green', linestyle='--')
    plt.axhline(y=quantile_25.mean()-1.96*quantile_25.std(), color='green', linestyle='--')
    threshold = quantile_25.mean()+1.96*quantile_25.std()
    plt.axvline(x =n -(quantile_25.sort_values() > threshold).sum(),
            linestyle='--')
    plt.legend()

    print(n -(quantile_25.sort_values() > threshold).sum())
    
    return quantile_25, quantile_75

In [ ]:
# quantile_25, quantile_75 = high_and_low_quantile(df, 'cpu')

In [ ]:
# dd = pd.DataFrame()
# dd['< quantile(.25), %'] = quantile_25.values
# dd['> quantile(.75), %'] = quantile_75.values

In [ ]:
# sns.jointplot(data=dd, x='< quantile(.25), %', y='> quantile(.75), %', kind='kde', size=8, aspect=2)

In [ ]:
# quantiles = pd.DataFrame()
# quantiles['q25'] = quantile_25.values
# quantiles['q75'] = quantile_75.values
# quantiles['ip'] = quantile_25.index

In [ ]:
# sns.regplot(data=quantiles, x='q25', y='q75')

## Выделим валидные и невалидные ip по отношению к квантилям

In [ ]:
valid_ip = unn[unn['class'] == 1]
invalid_ip = unn[unn['class'] == 0]

In [ ]:
invalid_ip.shape

In [ ]:
valid_ip.shape

In [ ]:
valid_ip.sort_values(by='nunique', ascending=False).head()

In [ ]:
fig = plt.figure(figsize=(17,9))
fig.suptitle(u'Распределение статистик для атрибута inUnit в разрезе разделенных на две части ip', fontsize=16)
plt.subplot(231)
sns.distplot(invalid_ip['nunique'])
plt.subplot(232)
sns.distplot(invalid_ip['n'])
plt.subplot(233)
sns.distplot(invalid_ip['ratio'])
plt.subplot(234)
sns.distplot(valid_ip['n'])
plt.subplot(235)
sns.distplot(valid_ip['nunique'])
plt.subplot(236)
sns.distplot(valid_ip['ratio'])

## Выделим ip  в группы

In [ ]:
un.head(2)

In [ ]:
ips_backets = pd.DataFrame()
ns = []
mean_ns = []
std_ns = []
median_ns = []
count_ns = []

nun = []
mean_nun = []
std_nun = []
median_nun = []
count_nun = []

nr = []
mean_nr = []
std_nr = []
median_nr = []

nns = np.linspace(1,un['n'].max(),50)
nuns = np.linspace(1,un['nunique'].max(),10)
indexs = []
for ix, nn in enumerate(nns[:-1]):
    for jx, nu in enumerate(nuns[:-1]):
        mask = (un['n'].isin(range(int(nns[ix]), int(nns[ix+1])))) & \
                (un['nunique'].isin(range(int(nuns[jx]), int(nuns[jx+1]))))
        indexs.append('{}-{}; {}-{}'.format(int(nns[ix]), int(nns[ix+1]), int(nuns[jx]), int(nuns[jx+1])))
        
        ns.append(un.loc[mask, 'n'].count())
        count_ns.append(un.loc[mask, 'n'].unique().shape[0])
        mean_ns.append(np.mean(un.loc[mask, 'n'].unique()))
        std_ns.append(np.std(un.loc[mask, 'n'].unique()))
        median_ns.append(np.median(un.loc[mask, 'n'].unique()))
        
        nun.append(un.loc[mask, 'nunique'].count())
        count_nun.append(un.loc[mask, 'nunique'].unique().shape[0])
        mean_nun.append(np.mean(un.loc[mask, 'nunique'].unique()))
        std_nun.append(np.std(un.loc[mask, 'nunique'].unique()))
        median_nun.append(np.median(un.loc[mask, 'nunique'].unique()))
        
#         nr.append(un.loc[mask, 'ratio'].count())
        mean_nr.append(np.mean(un.loc[mask, 'ratio'].unique()))
        std_nr.append(np.std(un.loc[mask, 'ratio'].unique()))
        median_nr.append(np.median(un.loc[mask, 'ratio'].unique()))

ips_backets['bucket'] = indexs
ips_backets['bucket_n'] = ips_backets['bucket'].apply(lambda x: x.split('; ')[0])
ips_backets['bucket_nun'] = ips_backets['bucket'].apply(lambda x: x.split('; ')[1])
        

ips_backets['n'] = ns    
ips_backets['unique_n'] = count_ns
ips_backets['mean_n'] = mean_ns
ips_backets['std_n'] = std_ns
ips_backets['median_n'] = median_ns

ips_backets['count_nunique'] = count_nun
ips_backets['mean_nunique'] = mean_nun
ips_backets['std_nunique'] = std_nun
ips_backets['median_nunique'] = median_nun


ips_backets['mean_ratio'] = mean_nr
ips_backets['std_ratio'] = std_nr
ips_backets['median_ratio'] = median_nr

ips_backets.drop('bucket', axis=1, inplace=True)

# ips_backets.index = indexs

In [ ]:
ips_backets.shape

In [ ]:
ips_backets.head(48)

## Интерполируем данные на равномерную сетку

In [ ]:
def create_traces_ts(ts_cpu):
    import datetime as dd
    ii = ts_cpu.index
    return traces.TimeSeries([
        (
            dd.datetime(ii[i].year, ii[i].month, ii[i].day, ii[i].hour, ii[i].minute, ii[i].second), 
            ts_cpu[i]
        ) for i in range(ts_cpu.shape[0])])

def interpolate_ts(ts, interval=30):
    import datetime as dd
    ii = ts.index
    ts = create_traces_ts(ts)
    begin = ii[0]
    print(begin)
    end = ii[-1]
    print(end)
    ss = ts.sample(
        sampling_period=dd.timedelta(minutes=interval),
        start=dd.datetime(begin.year, begin.month, begin.day, begin.hour),
        end=dd.datetime(end.year, end.month, end.day, end.hour),
        interpolate='linear',
    )
    return pd.Series(data=[i[1] for i in ss] , index=[i[0] for i in ss])

In [ ]:
valid_ip.ip.head(2)

In [ ]:
df.head()

In [ ]:
ip = valid_ip.ip.sample(36)

In [ ]:
subset = df[df['ip'].isin(ip)]

In [ ]:
subset.head()

In [ ]:
fig = plt.figure(figsize=(18,10))

# fig.suptitle(u'Характерный вид данных для cpuUtil')
for ix, i in enumerate(ip[:9]):
    ts = pd.Series(subset.loc[subset['ip'] ==i, 'out'].values, index=subset.loc[subset['ip'] == i, 'date'].values)
    m = ts.mean()
    d = ts.std()
    pylab.title(u'ip = {}; E={}; std={}'.format(i, round(m,3), round(d,3)))
    pylab.subplot(3,3,ix+1)
    plt.scatter(ts.index.values, ts.values)
    plt.xticks(rotation=40)
#     ts.plot(figsize=(18,13))
    plt.tight_layout()

# plt.plot()

In [ ]:
ip = '10.10.137.77'

In [ ]:
tt = pd.Series(subset.loc[subset['ip'] ==i, 'cpu'].values, index=subset.loc[subset['ip'] == i, 'date'].values)

In [ ]:
sns.distplot(tt)

In [ ]:
plt.scatter(tt.index.values, tt.values)
plt.xticks(rotation=70)

In [ ]:
tt.tail()

In [ ]:
tt[:1300].plot()

In [ ]:
tt.head(1)

In [ ]:
ts.tail(1)

In [ ]:
ts_inter = interpolate_ts(ts)

In [ ]:
ts_inter.plot()

In [ ]:
i

In [ ]:
ts_inter.index[:3]

In [ ]:
ts_inter.plot()

In [ ]:
def invboxcox(y,lmbda):
    # обрабтное преобразование Бокса-Кокса
    if lmbda == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(lmbda*y+1)/lmbda))

# print(df.cpu.astype(float))
new_ts, lmbda = scs.boxcox(ts_inter) 
plt.plot(new_ts)
print("Оптимальный параметр преобразования Бокса-Кокса: %f" % lmbda)

In [ ]:
new_ts = pd.Series(new_ts, index=ts_inter.index.values)

In [ ]:
def exp_smoothing(vals, alpha):
    result = [vals[0]] # first value is same as series
    for n in range(1, len(vals)):
        result.append(alpha * vals[n] + (1 - alpha) * result[n-1])
    return result

In [ ]:
rr = exp_smoothing(new_ts,0.5)

In [ ]:
plt.plot(rr[:100])

In [ ]:
d = adfuller(x=new_ts)

In [ ]:
result = adfuller(x=new_ts)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
new_ts = pd.Series(new_ts, index=ts_inter.index.values)

In [ ]:
new_ts.head()

In [ ]:
new_ts.rolling(window=24).mean().plot()

In [ ]:
ts_internal_48 = exp_smoothing(ts_inter, 0.3)

In [ ]:
plt.plot(ts_internal_48[:])

In [ ]:
t = subset.loc[subset['ip'] ==i, 'timestamp'].astype(int).values
y = ts.values

# frequency, power = ast_stats.LombScargle(t=t, y=y).autopower()

In [ ]:
frequency, power = ast_stats.LombScargle(y=ts_inter).autopower()

In [ ]:
frequency[:5]

In [ ]:
plt.plot(frequency, power)

In [ ]:
rr[:3]